In [ ]:
#@title { vertical-output: true }

import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color as skcolor
import random
from tqdm import tqdm as loading
import time

MAX_STRINGS = 3200
STRING_WIDTH = 1
STRING_TRANSPARENCY = 0.2
MAX_PIVOTS_X = MAX_PIVOTS_Y = 40
PIVOT_SIZE = 10
IMAGE_TRANSPARENCY = 0.4
SHOW_IMAGE = SHOW_PIVOTS = False

def processed(img):
  if len(img.shape) == 2:
    processed_image = img.astype(np.float32) / 255.0
  elif img.shape[-1] == 4:
    alpha = img[..., 3:].astype(np.float32) / 255.0
    rgb_values = img[..., :3].astype(np.float32)
    composited = rgb_values * alpha + 255.0 * (1.0 - alpha)
    processed_image = skcolor.rgb2gray(composited / 255.0)
  else:
    processed_image = skcolor.rgb2gray(img.astype(np.float32)) / 255.0

  return img, processed_image

def setup_graph(image):
  fig, ax = plt.subplots(figsize=(8, 8))
  if SHOW_IMAGE: ax.imshow(image, cmap='gray', alpha=IMAGE_TRANSPARENCY, vmin=0, vmax=1)
  ax.grid(which='both', color='black', linestyle='-', linewidth=0.3, alpha=IMAGE_TRANSPARENCY)

  ax.set_xticks([])
  ax.set_yticks([])
  ax.set_xticklabels([])
  ax.set_yticklabels([])

  return fig, ax

def get_pivots_of(dimensions):
  global MAX_PIVOTS_X, MAX_PIVOTS_Y, PIVOT_SIZE
  height, width = dimensions

  if MAX_PIVOTS_X == -1:
    MAX_PIVOTS_X = width
  if MAX_PIVOTS_Y == -1:
    MAX_PIVOTS_Y = height

  MAX_PIVOTS_X = min(MAX_PIVOTS_X, width)
  MAX_PIVOTS_Y = min(MAX_PIVOTS_Y, height)

  x_start, y_start = 0, 0
  x_end, y_end = width - 1, height - 1

  top_points = [(0, x) for x in np.linspace(x_start, x_end, MAX_PIVOTS_X)]
  bottom_points = [(y_end, x) for x in np.linspace(x_start, x_end, MAX_PIVOTS_X)]

  vertical_ys = np.linspace(y_start, y_end, MAX_PIVOTS_Y)[1:-1]

  left_points = [(y, 0) for y in vertical_ys]
  right_points = [(y, x_end) for y in vertical_ys]

  return np.array(top_points + bottom_points + left_points + right_points)

def draw_pivots(ax, pivots):
  for y, x in pivots:
    ax.plot(x, y, 'ro', markersize=PIVOT_SIZE, alpha=0.7)

def draw_line(ax, pivot1, pivot2, color='black', linewidth=STRING_WIDTH):
  y1, x1 = pivot1
  y2, x2 = pivot2
  ax.plot([x1, x2], [y1, y2], color=color, linewidth=linewidth, alpha=STRING_TRANSPARENCY)

def generate_candidates(pivot, pivots):
  return [p for p in pivots if not np.array_equal(p, pivot)]

from skimage.draw import line

def calculate_entropies(start_point, end_points, pixel_map):
  interpolated_paths = []
  entropies = []

  for end_point in end_points:
    y1, x1 = start_point
    y2, x2 = end_point

    num_points = int(np.hypot(y2 - y1, x2 - x1)) + 1
    ys = np.linspace(y1, y2, num=num_points)
    xs = np.linspace(x1, x2, num=num_points)
    coords = np.stack((ys, xs), axis=1).astype(int)

    interpolated_paths.append(coords)

    modified_map = pixel_map.copy()

    for y, x in coords:
      if 0 <= y < modified_map.shape[0] and 0 <= x < modified_map.shape[1]:
        modified_map[y, x] = 0.0

    entropy = np.sum(np.abs(modified_map - pixel_map))
    entropies.append(entropy)

  total_entropy = np.sum(entropies)

  if total_entropy > 0:
    probabilities = np.array(entropies) / total_entropy
  else:
    probabilities = np.full(len(entropies), 1 / len(entropies))

  probabilities = probabilities / np.sum(probabilities)

  return interpolated_paths, probabilities

def sample(optionsA, optionsB, distribution):
  index = np.random.choice(len(optionsA), p=distribution)
  return optionsA[index], optionsB[index]

def stringify(image_path="test.png"):
  start_time = time.time()

  image, pixel_map = processed(io.imread(image_path))
  fig, ax = setup_graph(pixel_map)
  pivots = get_pivots_of(pixel_map.shape)
  if SHOW_PIVOTS: draw_pivots(ax, pivots)

  current_pivot = pivots[0]

  for _ in loading(range(MAX_STRINGS), desc="Doing the magic"):
    end_pivots = generate_candidates(current_pivot, pivots)
    affected_maps, probabilities = calculate_entropies(current_pivot, end_pivots, pixel_map)
    sampled_map, sampled_pivot = sample(affected_maps, end_pivots, probabilities)

    draw_line(ax, current_pivot, sampled_pivot)
    pixel_map = sampled_map
    current_pivot = sampled_pivot

  end_time = time.time()
  print()
  plt.show()

  elapsed = end_time - start_time
  print(f"\nTotal time taken: {elapsed:.2f} seconds")

image = input("Enter image file name: ")
stringify(image)